In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks
remote = callbacks.RemoteMonitor(root='http://localhost:9000')
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from keras.utils.visualize_util import plot
from keras.optimizers import *
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import csv
from sklearn.metrics import log_loss
from PIL import Image
import os
import random
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.models import model_from_json
from keras.applications.resnet50 import preprocess_input, decode_predictions
from convnetskeras.customlayers import Softmax4D
import gc
from convnetskeras.convnets import preprocess_image_batch, convnet
from convnetskeras.imagenet_tool import synset_to_dfs_ids
import PIL
from PIL import Image,ImageFilter, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
from matplotlib.patches import Ellipse
import tqdm as tq
from numpy.linalg import eig, inv
import scipy
from scipy import ndimage
from skimage import data, io, filters
from skimage import data, img_as_float
from skimage import exposure
import math
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
import skimage

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
binary_folder_train = './binary_dataset/train/'
binary_folder_train_fish = binary_folder_train+'fish'
binary_folder_train_not_fish = binary_folder_train+'not_fish'
binary_folder_test = './binary_dataset/test/'

data_gen_args = dict(
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.,
    zoom_range=0.1,
    )

train_datagen = ImageDataGenerator(**data_gen_args)
validation_datagen = ImageDataGenerator(**data_gen_args)
test_datagen = ImageDataGenerator(**data_gen_args)

train_generator = train_datagen.flow_from_directory(binary_folder_train, batch_size=16)
validation_generator = validation_datagen.flow_from_directory(binary_folder_train, batch_size=16)
test_generator = validation_datagen.flow_from_directory(binary_folder_test, batch_size=16)

Found 98943 images belonging to 2 classes.
Found 98943 images belonging to 2 classes.
Found 11800 images belonging to 2 classes.


In [ ]:
from keras.layers import merge, Input
def return_model():
    model = Sequential()
    model.add(Activation(activation="relu", input_shape=(3,None,None)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same', activation='relu'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same'))
    model.add(Dropout(0.2))

    model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(GlobalMaxPooling2D())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(2))
    model.add(Activation('softmax'))
    return model

Using Theano backend.


In [ ]:
def return_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False)

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    x = Dense(256,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    x = Dense(256,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    x = Dense(128,init='normal', activation='relu')(x)
    x = Dropout(0.005)(x)
    predictions = Dense(2,init='normal', activation='softmax')(x)

    model = Model(input=base_model.input, output=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    return model

In [ ]:
#dir(model)

In [ ]:
model = return_resnet50()
#model.summary()

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)

opt = SGD(lr=1e-1, decay=1e-6, momentum=0.2, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# train the model on the new data for a few epochs
nb_epoch = 100
model.fit_generator(train_generator,
        samples_per_epoch=1024,
        nb_epoch=nb_epoch,
        validation_data=validation_generator,
        verbose=0,
        nb_val_samples=256, callbacks=[remote])

In [ ]:
model.evaluate_generator(test_generator,512)

In [ ]:
model.save('model_2c_100e_R50_2')

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model_2c_100e_R50_2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_2c_100e_R50_2.h5")
print("Saved model to disk")


In [ ]:
import numpy as np
from keras.layers.core import  Lambda, Merge
from keras.layers.convolutional import Convolution2D
from keras import backend as K
from keras.engine import Layer
from os import listdir
from os.path import isfile, join, dirname
from scipy.io import loadmat
import gc
from keras.utils.layer_utils import layer_from_config
from keras.models import Model
from keras.layers import *

# Credits to heuritech for their great code which was a great inspiration.
# Some of the code comes directly from their repository.
# You can look it up: https://github.com/heuritech/convnets-keras


def depthfirstsearch(id_, out=None):
    if out is None:
        out = []
    if isinstance(id_, int):
        pass
    else:
        id_ = next(int(s[0]) for s in synsets if s[1][0] == id_)
        
    out.append(id_)
    children = synsets[id_-1][5][0]
    for c in children:
        depthfirstsearch(int(c), out)
    return out
	
# This is to find all the outputs that correspond to the class we want.
def synset_to_dfs_ids(synset):
    ids = [x for x in depthfirstsearch(synset) if x <= 1000]
    ids = [corr[x] for x in ids]
    return ids
	
# Keras doesn't have a 4D softmax. So we need this.
class Softmax4D(Layer):
    def __init__(self, axis=-1,**kwargs):
        self.axis=axis
        super(Softmax4D, self).__init__(**kwargs)

    def build(self,input_shape):
        pass

    def call(self, x,mask=None):
        e = K.exp(x - K.max(x, axis=self.axis, keepdims=True))
        s = K.sum(e, axis=self.axis, keepdims=True)
        return e / s

    def get_output_shape_for(self, input_shape):
        return input_shape
		

def get_dim(model, layer_index, input_shape=None):
    
    # Input shape is the shape of images used during training.
    if input_shape is not None:
        dummy_vector = np.zeros((1,) + input_shape)
    else:
        if model.layers[0].input_shape[2] is None:
            raise ValueError('You must provide \"input_shape = (3,256,256)\" for example when calling the function.')
        dummy_vector = np.zeros((1,) + model.layers[0].input_shape[1:])
    
    intermediate_layer_model = Model(input=model.input,
                                 output=model.layers[layer_index].output)
    
    out = intermediate_layer_model.predict(dummy_vector)
    
    return out.shape[1:]
	

def from_config(layer, config_dic):
    config_correct = {}
    config_correct['class_name'] = type(layer)
    config_correct['config'] = config_dic
    return layer_from_config(config_correct)
	

def add_to_model(x, layer):
    new_layer = from_config(layer, layer.get_config())
    x = new_layer(x)
    if layer.get_weights() is not None:
        new_layer.set_weights(layer.get_weights())
    return x
	

def layer_type(layer):
    return str(layer)[10:].split(" ")[0].split(".")[-1]
	

def detect_configuration(model):
    # must return the configuration and the number of the first pooling layer
    
    # Names (types) of layers from end to beggining
    inverted_list_layers = [layer_type(layer) for layer in model.layers[::-1]]
    
    layer1 = None
    layer2 = None 
    
    i = len(model.layers)
    
    for layer in inverted_list_layers:
        i -= 1
        if layer2 is None:
            if layer == "GlobalAveragePooling2D" or layer == "GlobalMaxPooling2D":
                layer2 = layer

            elif layer == "Flatten":
                return "local pooling - flatten", i-1
            
        else:
            layer1 = layer
            break
            
    if layer1 == "MaxPooling2D" and layer2 == "GlobalMaxPooling2D":
        return "local pooling - global pooling (same type)", i
    elif layer1 == "AveragePooling2D" and layer2 == "GlobalAveragePooling2D":
        return "local pooling - global pooling (same type)", i
    
    elif layer1 == "MaxPooling2D" and layer2 == "GlobalAveragePooling2D":
        return "local pooling - global pooling (different type)", i+1
    elif layer1 == "AveragePooling2D" and layer2 == "GlobalMaxPooling2D":
        return "local pooling - global pooling (different type)", i+1
    
    else:
        return "global pooling", i
		
    
def add_zeros(w, nb_zeros):
    
    n = w.shape[3]
    indexes = np.array(range(1, n))
    w1 = w
    for i in range(nb_zeros):
        w1 = np.insert(w1, indexes + i, 0, axis=2)
    for i in range(nb_zeros):
        w1 = np.insert(w1, indexes + i, 0, axis=3)
    return w1
	
    
def insert_weights(layer, new_layer):
    W,b = layer.get_weights()
    n_filter,previous_filter,ax1,ax2 = new_layer.get_weights()[0].shape
    ax1 = ax2 = int(np.sqrt(layer.get_weights()[0].shape[0]/new_layer.get_weights()[0].shape[1]))
    new_W = W.reshape((previous_filter,ax1,ax2,n_filter))
    new_W = new_W.transpose((3,0,1,2))
    new_W = new_W[:,:,::-1,::-1]
	
    
    if ax1!=1:
        insert_zeros = int((new_layer.get_weights()[0].shape[2] - ax1)/(ax1-1))
        print("insert_zeros=" + str(insert_zeros))
        new_W =  add_zeros(new_W, insert_zeros)
    
    new_layer.set_weights([new_W,b])
	
    
def copy_last_layers(model, begin,x):
    
    i=begin
    
    for layer in model.layers[begin:]:
        if layer_type(layer) == "Dense":
            
            if i == len(model.layers)-1:
                x = add_reshaped_layer(layer,x,1, no_activation=True)
            else:
                x = add_reshaped_layer(layer,x,1)
            
        elif layer_type(layer) == "Dropout":
            pass
                
        elif layer_type(layer) == "Activation" and i == len(model.layers)-1:
            break
               
        else:
            x = add_to_model(x, layer)
        i+=1
    
    x = Softmax4D(axis=1,name="softmax")(x)
    return x
    
                
def add_reshaped_layer(layer, x, size, no_activation=False, add_zeros = None):

    conf = layer.get_config()
    
    if no_activation:
        activation="linear"
    else:
        activation=conf["activation"]
        
    #size = int(np.sqrt(layer.get_weights()[0].shape[0]/conf["output_dim"]))
    
    new_layer = Convolution2D(conf["output_dim"],size,size, activation=activation, name=conf['name'])
         
        
    x= new_layer(x)
    # We transfer the weights:
    insert_weights(layer, new_layer)
    return x
    

def to_heatmap(model, input_shape = None, delete = False):
    
    # there are four configurations possible:
    # global pooling
    # local pooling - flatten
    # local pooling - global pooling (same type)
    # local pooling - global pooling (different type)
    
    model_type, index = detect_configuration(model)
    
    print("Model type detected: " + model_type)
    
    #new_layer.set_weights(model.layers[0].get_weights())
    img_input = Input(shape=(3,None,None))
   
    # Inchanged part:
    middle_model = Model(input=model.layers[1].input, output=model.layers[index-1].output)
    
    x = middle_model(img_input)
    
    print("Model cut at layer: " + str(index))
        
    if model_type == "global pooling":
        x = copy_last_layers(model, index+1,x)
              
    elif model_type == "local pooling - flatten":
        
        layer = model.layers[index]
        dic = layer.get_config()
        add_zeros = dic["strides"][0] - 1
        dic["strides"] = (1,1)
        new_pool = from_config(layer, dic)
        x = new_pool(x)
        
        size = get_dim(model, index, input_shape)[1]
        print("Pool size infered: " + str(size))
        
        conv_size = size + (size-1) * add_zeros
        
        print("New convolution size: " + str(conv_size))
        
        if index+2 != len(model.layers)-1:
            x = add_reshaped_layer(model.layers[index+2],x,conv_size, add_zeros=add_zeros)
        else:
            x = add_reshaped_layer(model.layers[index+2],x,conv_size, add_zeros=add_zeros,no_activation=True)
            
        x = copy_last_layers(model, index+3,x)
        
        
    elif model_type == "local pooling - global pooling (same type)":
        
        
        dim = get_dim(model, index, input_shape=input_shape)

        new_pool_size = model.layers[index].get_config()["pool_size"][0] * dim[1]
        
        print("Pool size infered: " + str(new_pool_size))
        
        x = AveragePooling2D(pool_size=(new_pool_size, new_pool_size), strides=(1,1)) (x)
        x = copy_last_layers(model, index+2,x)
        
        
    elif model_type == "local pooling - global pooling (different type)":
        x= copy_last_layers(model, index+1,x)
    else:
        raise IndexError("no type for model: " + str(model_type))
        
    
    
    if delete:
        del(model)
        gc.collect()
        print("Original model was deleted.")
    
    return Model(img_input, x)

In [ ]:
# load json and create model
json_file = open('model_2c_10e_R50_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model_2c_10e_R50_1.h5")
print("Loaded model from disk")

In [ ]:
new_model = to_heatmap(model, input_shape = (3,256,256), delete=False)

In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
res_model = ResNet50(weights='imagenet', include_top=True,input_shape = (3,224,224))
new_res_model = to_heatmap(res_model, input_shape = (3,224,224), delete=False)
new_res_model.compile(optimizer=sgd, loss='mse')

In [ ]:
root_path = "./train/train/ALB/"
file = os.listdir(root_path)
N = int(np.random.uniform(0,1)*len(file))
print(N)
plt.figure(0)
img_path = root_path+file[N]
img = image.load_img(img_path)
plt.imshow(img)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

out = new_model.predict(x)
plt.figure(1)
heatmap = out[0,[1]].sum(axis=0)
#p2, p98 = np.percentile(heatmap, (2, 98))
#heatmap = exposure.rescale_intensity(heatmap_out, in_range=(p2, p98))
plt.imshow(heatmap, interpolation="none")
plt.show()

In [ ]:
def filter_mask_path(img_path,th,model_name):
    def binarize(value, threshold):
        if value < threshold:
            return 255
        else:
            return 0
    json_file = open(model_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(model_name+".h5")
    print("Loaded model from disk")
    img = image.load_img(img_path)
    plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    out = new_model.predict(x)
    heatmap = out[0,[1]].sum(axis=0)
    b = np_binarize(heatmap, th)
    import scipy
    size = np.array(img.size)
    b = scipy.misc.imresize(b, [size[1],size[0]], interp='bilinear', mode=None)
    img_array = np.array(img)
    bb = b >= 2
    mask = np.zeros_like(img_array)
    for i in range(3):
        mask[:,:,i] = bb
    idx=(mask==0)
    img_array[idx]=mask[idx]
    img_filter = Image.fromarray(img_array)
    return img_filter

In [ ]:
def binarize(value, threshold,state=True):
    if state:
        if value < threshold:
            return 255
        else:
            return 0
    else:
        if value < threshold:
            return 0
        else:
            return 255        
def filter_mask(img,heatmap,th,state):
    np_binarize = np.vectorize(binarize)
    bb = np_binarize(heatmap, th,state)
    import scipy
    size = np.array(img.size)
    b = scipy.misc.imresize(bb, [size[1],size[0]], interp='bicubic', mode=None)
    img_array = np.array(img)
    bb = b >= 2
    mask = np.zeros_like(img_array)
    for i in range(3):
        mask[:,:,i] = bb
    idx=(mask==0)
    img_array[idx]=mask[idx]
    img_filter = Image.fromarray(img_array)
    return img_filter,Image.fromarray(b)

In [ ]:
plt.figure()
plt.imshow(img)
#img_f = filter_mask(img_path,0.999,'model_2c_100e_R50_2')
img_f,b1 = filter_mask(img,heatmap,0.999,True)
#img_f = filter_mask(img_f,heatmap2,0.999)
plt.figure()
plt.imshow(img_f)
plt.show()

In [ ]:
from convnetskeras.imagenet_tool import synset_to_dfs_ids
x = image.img_to_array(img_f)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
out = new_res_model.predict(x)

s = "n02512053"
ids = synset_to_dfs_ids(s)
heatmap_out = out[0,ids].sum(axis=0)
#p2, p98 = np.percentile(heatmap_out, (2, 98))
#heatmap_out = exposure.rescale_intensity(heatmap_out, in_range=(p2, p98))
plt.figure()
plt.imshow(heatmap_out, interpolation='none')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
out = new_res_model.predict(x)

s = "n02512053"
ids = synset_to_dfs_ids(s)
heatmap_out2 = out[0,ids].sum(axis=0)
#p2, p98 = np.percentile(heatmap_out, (2, 98))
#heatmap_out = exposure.rescale_intensity(heatmap_out, in_range=(p2, p98))
plt.figure()
plt.imshow(heatmap_out2, interpolation='none')
plt.show()   

In [ ]:
plt.figure()
plt.imshow(img_f)
#img_f = filter_mask(img_path,0.999,'model_2c_100e_R50_2')
img_f2,b2 = filter_mask(img_f,heatmap_out,0.0001,False)
#img_f = filter_mask(img_f,heatmap2,0.1)
plt.figure()
plt.imshow(b2)
plt.figure()
plt.imshow(img_f2)
plt.show()

In [ ]:
def change_class(value, old, new):
    if value == old:
        return new
    else:
        return value
np_change_class = np.vectorize(change_class)




def stuff(mat,i,j,islands_list, count, islands_matrix):
    if mat[i][j] == 0:
        pass
    else:
        before = int(islands_matrix[i][j-1])
        above = int(islands_matrix[i-1][j])
        if before == 0:
            if  above== 0:
                islands_matrix[i][j] = count
                count +=1
                islands_list.append([(j,i)])
            else: 
                islands_matrix[i][j] = above
                islands_list[above-1].append((j,i))
        
        else:
            if above == 0:
                islands_matrix[i][j] = before
                islands_list[before-1].append((j,i))
            
            
            else:
                if above == before:
                    islands_matrix[i][j] = before
                    islands_list[before-1].append((j,i))
                
                #it's on.
                else:
                    old = max((above,before))
                    new = min((above,before))
                    
                    islands_matrix[i][j] = new
                    islands_matrix = np_change_class(islands_matrix, old, new)
                    islands_list[new-1] += islands_list[old-1]
                    islands_list[new-1].append((j,i))
                    islands_list[old-1] = []
                    
    return islands_list, count, islands_matrix        


# The not fun part
def group_by_island(matrix):
    
    h, l = np.shape(matrix)
    border = np.array([[0]*(l)])
    mat = np.concatenate((border,matrix), axis=0)
    border = np.array([[0]*(h+1)])
    mat = np.concatenate((border.T, mat), axis=1)
    
    islands_matrix =np.zeros((h+1,l+1))
    count = 1
    islands_list = []
    
    for i in range(1,h+1):
        for j in range(1, l+1):
            islands_list, count, islands_matrix = stuff(mat,i,j,islands_list, count, islands_matrix)

    
    
    return islands_matrix, islands_list
    

In [ ]:
islands_matrix, islands_list = group_by_island(b2)
plt.imshow(islands_matrix, interpolation='none')
plt.show()

In [ ]:
def find_extremes(list_coordinates):
    max_x = max(list_coordinates, key = lambda x: x[0])[0]
    min_x = min(list_coordinates, key = lambda x: x[0])[0]
    max_y = max(list_coordinates, key = lambda x: x[1])[1]
    min_y = min(list_coordinates, key = lambda x: x[1])[1]
    
    #print(min_x, min_y, max_x, max_y )
    
    return min_x - 1, min_y - 1, max_x-min_x + 1, max_y-min_y + 1


def get_rectangles(islands_list, area_limit=4):
    list_coordinates = []
    for island in islands_list:
        if len(island) == 0:
            continue
        else:
            coordinates = find_extremes(island)
            #print(coordinates)
            #print(coordinates[2]*coordinates[3])
            if coordinates[2]*coordinates[3] >=area_limit:
                #print(coordinates)
                list_coordinates.append(coordinates)
            
    return list_coordinates
    

In [ ]:
rectangles = get_rectangles(islands_list)

In [ ]:
rectangles

In [ ]:
Fish_th = 0.99
for rect in rectangles:
    x = rect[0]
    y = rect[1]
    w = rect[2]
    h = rect[3]
    img_crop = img.crop((x,y,x+w,y+h))
    plt.figure()
    plt.imshow(img_crop)
plt.show()

In [ ]:
def get_min_dim(size):
    w,h = size
    while w > 150 or h > 150:
        w/=2
        h/=2
    print(w,h)
    return int(w),int(h)
    

In [ ]:
import matplotlib.pyplot as plt

from skimage import data, color
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
size = img_crop.size
img_to_use = img_crop.resize(get_min_dim(size), PIL.Image.ANTIALIAS)
plt.imshow(img_to_use)

In [ ]:
# Load picture, convert to grayscale and detect edges
image_rgb = np.array(img_to_use)
image_gray = color.rgb2gray(image_rgb)
edges = filters.scharr(image_gray)
p2, p98 = np.percentile(edges, (1, 99))
img_rescale = exposure.rescale_intensity(edges, in_range=(p2, p98))
plt.imshow(edges)
th = 0.6
image_0 = []
Max = 0
for line in img_rescale:
    obj = []
    for y in line:
        if y > Max:
            Max = y
        if y > th:
            obj.append(255)
        else:
            obj.append(0)    
    image_0.append(obj)
image_0 = Image.fromarray(np.array(image_0))
#image_1 = ndimage.binary_erosion(image_0,structure=np.ones((2,2)))
plt.figure(4)
plt.imshow(image_0)
plt.show()

In [ ]:

# Perform a Hough Transform
# The accuracy corresponds to the bin size of a major axis.
# The value is chosen in order to get a single high accumulator.
# The threshold eliminates low accumulators
result = hough_ellipse(np.array(image_0), accuracy=20, threshold=10,
                       min_size=10, max_size=50)
result.sort(order='accumulator')

# Estimated parameters for the ellipse
print(result)
best = list(result[-1])
yc, xc, a, b = [int(round(x)) for x in best[1:5]]
orientation = best[5]

# Draw the ellipse on the original image
cy, cx = ellipse_perimeter(yc, xc, a, b, orientation)
image_rgb[cy, cx] = (0, 0, 255)
# Draw the edge (white) and the resulting ellipse (red)
edges = color.gray2rgb(edges)
edges[cy, cx] = (250, 0, 0)

fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8, 4), sharex=True,
                                sharey=True,
                                subplot_kw={'adjustable':'box-forced'})

ax1.set_title('Original picture')
ax1.imshow(image_rgb)

ax2.set_title('Edge (white) and result (red)')
ax2.imshow(edges)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
def get_mask_from_img(classe,img_name,size):
    import os
    import collections
    import json
    import numpy as np
    json_name = classe.lower()+'_labels.json'
    data_json = collections.defaultdict(int)
    with open(json_name,'r+') as f:
        labels = json.load(f)
        for label in labels:
            name = label['filename'].split('/')[-1]
            if img_name == name:
                rects = label['annotations']
                mask = np.zeros((100,100))
                if len(rects) > 0 :
                    x_r = 100/size[0]
                    y_r = 100/size[1]
                    for rect in rects:
                        x = rect['x']*x_r
                        y = rect['y']*y_r
                        w = rect['width']*x_r
                        h = rect['height']*y_r
                        for n in range(size[0]-1):
                            for k in range(size[1]-1):
                                if n > x and n < (x+w) and k > y and k < (y+h) and n < 100 and k < 100:
                                        mask[k][n] = 1
                   # img_array[idx]=mask[idx]
                    return mask
                else:
                    return mask    
plt.imshow(get_mask_from_img('ALB','img_00010.jpg',(1280,720)))
plt.show()